#### CREATE STREAMING TABLE FOR `ADDRESS.CSV` AND LOAD INTO `BRONZE_ADDRESS`

In [0]:
CREATE OR REFRESH STREAMING TABLE BRONZE_ADDRESSES
COMMENT 'Raw address data ingested from cloud storage volume'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT
*,
_metadata.file_path AS file_path,
_metadata.file_name AS file_name,
CURRENT_TIMESTAMP AS load_timestamp
FROM
cloud_files('/Volumes/circuitbox_dev/landing/operational_data/addresses/', 
'csv',
map('cloudFiles.inferColumnTypes', 'true')
)

#### PERFORM DATA QUALITY CHECKS
- FAIL IS `CUSTOMER ID` IS NULL
- DROP RECORDS WHEN `CUSTOMER NAME` IS NULL
- WARN IF 'TELEPHONE NUMBER' IS LESS THAN 10 CHARACTERS
- WARN IF `EMAIL` IS NULL
- WARN IF `DOB` IS BEFORE 1920

In [0]:
CREATE OR REFRESH STREAMING TABLE SILVER_ADDRESSES
(
CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
CONSTRAINT valid_address_line_1 EXPECT (address_line_1 IS NOT NULL) ON VIOLATION DROP ROW,
CONSTRAINT valid_postcode EXPECT (LENGTH(postcode) >= 5)
)
COMMENT 'Cleaned address data ingested from cloud storage volume'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT
customer_id,
address_line_1,
city,
state,
postcode,
CAST(created_date AS DATE) AS created_date
FROM
STREAM(LIVE.BRONZE_ADDRESSES);

#### PERFORM SLOWLY CHANGING DIMENSION (SCD I) FOR `ADRESSES`

In [0]:
CREATE OR REFRESH STREAMING TABLE SCD_1_ADDRESSES
COMMENT 'SCD TYPE 1'
TBLPROPERTIES ('quality' = 'gold')

In [0]:
APPLY CHANGES INTO LIVE.SCD_1_ADDRESSES
FROM STREAM(LIVE.SILVER_ADDRESSES)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1;

#### PERFORM SLOWLY CHANGING DIMENSION (SCD 2) FOR `ADDRESSES`

In [0]:
CREATE OR REFRESH STREAMING TABLE SCD_2_ADDRESSES
COMMENT 'SCD TYPE 2'
TBLPROPERTIES ('quality' = 'gold')

In [0]:
APPLY CHANGES INTO LIVE.SCD_2_ADDRESSES
FROM STREAM(LIVE.SILVER_ADDRESSES)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 2;